In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
import string
import xgboost as xgb
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn import ensemble, metrics, model_selection, naive_bayes
color = sns.color_palette()

eng_stopwords = set(stopwords.words("english"))
pd.options.mode.chained_assignment = None

from sklearn.model_selection import KFold,cross_val_score
%matplotlib inline

from sklearn.metrics import mean_squared_error

from sklearn import model_selection, preprocessing, metrics
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import lightgbm as lgb

from sklearn.ensemble import  RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression, Ridge,RidgeCV,LarsCV,LassoCV,ElasticNetCV,LassoLarsCV
!ls ../input/mh-the-price-of-books/

In [ ]:
train = pd.read_excel('../input/mh-the-price-of-books/Data_Train.xlsx')
test = pd.read_excel('../input/mh-the-price-of-books/Data_Test.xlsx')
ss = pd.read_excel('../input/mh-the-price-of-books/Sample_Submission.xlsx')

In [ ]:
train.shape,test.shape,ss.shape

In [ ]:
sns.distplot(np.log(train['Price']))

In [ ]:
for i in test.columns:
    print(i,':',train[i].nunique())

In [ ]:
train['BookCategory'].value_counts().index

In [ ]:
l1 = ['Action & Adventure', 'Crime, Thriller & Mystery',
       'Biographies, Diaries & True Accounts',
       'Language, Linguistics & Writing', 'Comics & Mangas', 'Romance',
       'Humour', 'Arts, Film & Photography',
       'Computing, Internet & Digital Media', 'Sports', 'Politics']
l2 = list(range(len(l1)))
d = dict(zip(l1,l2))

In [ ]:
train['BookCategory'] = train['BookCategory'].map(d)
test['BookCategory'] = test['BookCategory'].map(d)

In [ ]:
train = pd.concat([train,pd.get_dummies(train['BookCategory'],prefix ='BookCategory',drop_first=True)],axis=1)
test = pd.concat([test,pd.get_dummies(test['BookCategory'],prefix ='BookCategory',drop_first=True)],axis=1)
                  

In [ ]:
train.drop('BookCategory',axis=1,inplace=True)
test.drop('BookCategory',axis=1,inplace=True)

In [ ]:
#train['Reviews'].value_counts()

In [ ]:
#train['Ratings'].value_counts()

In [ ]:
train['Title'].value_counts()

In [ ]:
train[train.Title == 'A Game of Thrones (A Song of Ice and Fire)']

In [ ]:
train['reviews'] = train['Reviews'].apply(lambda x: x.split()[0])
test['reviews'] = test['Reviews'].apply(lambda x: x.split()[0])

train['ratings'] = train['Ratings'].apply(lambda x: x.split()[0])
test['ratings'] = test['Ratings'].apply(lambda x: x.split()[0])

train.drop(['Reviews','Ratings'],axis=1,inplace=True)
test.drop(['Reviews','Ratings'],axis=1,inplace=True)

In [ ]:
## Number of words in the text ##
train["num_words"] = train["Synopsis"].apply(lambda x: len(str(x).split()))
test["num_words"] = test["Synopsis"].apply(lambda x: len(str(x).split()))

## Number of unique words in the text ##
train["num_unique_words"] = train["Synopsis"].apply(lambda x: len(set(str(x).split())))
test["num_unique_words"] = test["Synopsis"].apply(lambda x: len(set(str(x).split())))

## Number of characters in the text ##
train["num_chars"] = train["Synopsis"].apply(lambda x: len(str(x)))
test["num_chars"] = test["Synopsis"].apply(lambda x: len(str(x)))

## Number of stopwords in the text ##
train["num_stopwords"] = train["Synopsis"].apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))
test["num_stopwords"] = test["Synopsis"].apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))

## Number of punctuations in the text ##
train["num_punctuations"] =train['Synopsis'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]) )
test["num_punctuations"] =test['Synopsis'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]) )

## Number of title case words in the text ##
train["num_words_upper"] = train["Synopsis"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))
test["num_words_upper"] = test["Synopsis"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))

## Number of title case words in the text ##
train["num_words_title"] = train["Synopsis"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))
test["num_words_title"] = test["Synopsis"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))

## Average length of the words in the text ##
train["mean_word_len"] = train["Synopsis"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
test["mean_word_len"] = test["Synopsis"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

In [ ]:
train['Edition_cover'] = train['Edition'].apply(lambda x : x.split(',')[0])
test['Edition_cover'] = test['Edition'].apply(lambda x : x.split(',')[0])

In [ ]:
def rare_imputation(X_train, X_test, variable):
    
    # find the most frequent category
    frequent_cat = X_train.groupby(variable)[variable].count().sort_values().tail(1).index.values[0]
    
    # find rare labels
    temp = X_train.groupby([variable])[variable].count()/np.float(len(X_train))
    rare_cat = [x for x in temp.loc[temp<0.007].index.values]
    
    # create new variables, with Rare labels imputed
    
    # by the most frequent category
    X_train[variable] = np.where(X_train[variable].isin(rare_cat), frequent_cat, X_train[variable])
    X_test[variable] = np.where(X_test[variable].isin(rare_cat), frequent_cat, X_test[variable])
    
    # by adding a new label 'Rare'
#     X_train[variable+'_rare_imp'] = np.where(X_train[variable].isin(rare_cat), 99999, X_train[variable])
#     X_test[variable+'_rare_imp'] = np.where(X_test[variable].isin(rare_cat), 99999, X_test[variable])

In [ ]:
rare_imputation(train, test, 'Edition_cover')

In [ ]:
train['Edition_cover'].value_counts()

In [ ]:
test['Edition_cover'].value_counts()

In [ ]:
d = {'Paperback':'Paperback', 'Hardcover':'Hardcover', 'Mass Market Paperback':'Mass Market Paperback', '(Chinese)':'Mass Market Paperback'}
test['Edition_cover'] = test['Edition_cover'].map(d)

In [ ]:
train = pd.concat([train,pd.get_dummies(train['Edition_cover'],prefix ='Edition_cover',drop_first=True)],axis=1)
test = pd.concat([test,pd.get_dummies(test['Edition_cover'],prefix ='Edition_cover',drop_first=True)],axis=1)

train.drop('Edition_cover',axis=1,inplace=True)
test.drop('Edition_cover',axis=1,inplace=True)

In [ ]:
train.head()

In [ ]:
train['Edition_date'] = train['Edition'].apply(lambda x: x.split()[-1])
test['Edition_date'] = test['Edition'].apply(lambda x: x.split()[-1])

In [ ]:
train['Edition_date'].value_counts().index

In [ ]:
l1 = ['2018', '2017', '2016', '2015', '2014', '2013', '2019', '2012', '2011',
       '2010', '2009', '2008', '2005', '2006', '2007', '2003', '2004', '2002',
       '2000', '2001', '1999', '1997', '1994', '1998', '1992', '1995', '1996',
       '1993', '1989', '1991', '1986', '1988', '1984', '1990', '1982', '1987',
       '1985', '1983', 'Import', '1976', 'set', '1980', '1971', '1975', '1974',
       '1977', '1964', 'Print', '1979', '1960', '1978', '1961', '1969', 'NTSC',
       '1900', '1925', 'Audiobook', 'Facsimile', '1970', 'Edition',
       'Unabridged', '1973', '1981', '1905']
l2 = [2018, 2017, 2016, 2015, 2014, 2013, 2019, 2012, 2011,
       2010, 2009, 2008, 2005, 2006, 2007, 2003, 2004, 2002,
       2000, 2001, 1999, 1997, 1994, 1998, 1992, 1995, 1996,
       1993, 1989, 1991, 1986, 1988, 1984, 1990, 1982, 1987,
       1985, 1983, 9999, 1976, 9999, 1980, 1971, 1975, 1974,
       1977, 1964, 9999, 1979, 1960, 1978, 1961, 1969, 9999,
       1900, 1925, 9999, 9999, 1970, 9999,
       9999, 1973, 1981, 1905]

In [ ]:
d = dict(zip(l1,l2))

In [ ]:
train['Edition_date'] = train['Edition_date'].map(d)
test['Edition_date'] = test['Edition_date'].map(d)

In [ ]:
#train['Edition_date'].value_counts()

In [ ]:
train['Genre'].value_counts()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(ngram_range=(1, 1), lowercase=False)
train_route = tf.fit_transform(train['Genre'])
test_route = tf.transform(test['Genre'])

train_route = pd.DataFrame(data=train_route.toarray(), columns=tf.get_feature_names())
test_route = pd.DataFrame(data=test_route.toarray(), columns=tf.get_feature_names())

train_route.shape,test_route.shape

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(ngram_range=(1, 1), lowercase=False)
train_route1 = tf.fit_transform(train['Title'])
test_route1 = tf.transform(test['Title'])

train_route1 = pd.DataFrame(data=train_route1.toarray(), columns=tf.get_feature_names())
test_route1 = pd.DataFrame(data=test_route1.toarray(), columns=tf.get_feature_names())

train_route1.shape,test_route1.shape

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
tf = TfidfVectorizer(ngram_range=(1, 5), lowercase=False)
train_route2 = tf.fit_transform(train['Synopsis'])
test_route2 = tf.transform(test['Synopsis'])

train_route2 = pd.DataFrame(data=train_route2.toarray(), columns=tf.get_feature_names())
test_route2 = pd.DataFrame(data=test_route2.toarray(), columns=tf.get_feature_names())

train_route2.shape,test_route2.shape

tf = CountVectorizer(ngram_range=(1, 5), lowercase=False)
train_route3 = tf.fit_transform(train['Synopsis'])
test_route3 = tf.transform(test['Synopsis'])

train_route3 = pd.DataFrame(data=train_route3.toarray(), columns=tf.get_feature_names())
test_route3 = pd.DataFrame(data=test_route3.toarray(), columns=tf.get_feature_names())

train_route3.shape,test_route3.shape

In [ ]:
# custom function to run light gbm model
def run_lgb(train_X, train_y, val_X, val_y, test_X):
    params = {
        "objective" : "regression",
        "metric" : "rmse", 
        "num_leaves" : 30,
        "min_child_samples" : 100,
        "learning_rate" : 0.1,
        "bagging_fraction" : 0.7,
        "feature_fraction" : 0.5,
        "bagging_frequency" : 5,
        "bagging_seed" : 2018,
        "verbosity" : -1
    }
    
    params ={'learning_rate': 0.04, 'max_depth': 15, 'min_child_samples': 8, 'num_leaves': 55,"objective" : "regression", "metric" : "rmse"}
    
    lgtrain = lgb.Dataset(train_X, label=train_y)
    lgval = lgb.Dataset(val_X, label=val_y)
    model = lgb.train(params, lgtrain, 1000, valid_sets=[lgval], early_stopping_rounds=100, verbose_eval=100)
    
    pred_test_y = model.predict(test_X, num_iteration=model.best_iteration)
    pred_val_y = model.predict(val_X, num_iteration=model.best_iteration)
    return pred_test_y, model, pred_val_y

In [ ]:
train_df = pd.concat([train,train_route,train_route1,train_route2,train_route3],axis=1)
test_df = pd.concat([test,test_route,test_route1,test_route2,test_route3],axis=1)


In [ ]:
train_df.shape,test_df.shape

In [ ]:
cat_cols = list(train_df.select_dtypes(include=['object']).columns)
cat_cols

In [ ]:
train_df[cat_cols].head()

In [ ]:
drop_col = ['Title','Edition','Synopsis','Genre']
train_df.drop(drop_col,axis=1,inplace=True)
test_df.drop(drop_col,axis=1,inplace=True)

In [ ]:
train_df1 = train_df[['Author','ratings','reviews']].copy()
test_df1 = test_df[['Author','ratings','reviews']].copy()
train_df.drop('Author',axis=1,inplace=True)
test_df.drop('Author',axis=1,inplace=True)

In [ ]:
test_df1.head()

In [ ]:
train_df1 = train_df1.iloc[:,[1,3,4]]
test_df1 = test_df1.iloc[:,[1,3,4]]

In [ ]:
train_df1['reviews'] = train_df1['reviews'].apply(lambda x : float(x))
test_df1['reviews'] = test_df1['reviews'].apply(lambda x : float(x))

In [ ]:
train_df1['ratings'] = train_df1['ratings'].apply(lambda x : x.replace(',',''))
test_df1['ratings'] = test_df1['ratings'].apply(lambda x : x.replace(',',''))

train_df1['ratings'] = train_df1['ratings'].apply(lambda x : int(x))
test_df1['ratings'] = test_df1['ratings'].apply(lambda x : int(x))

In [ ]:
train_df1.head()

In [ ]:
train_df.drop('ratings	reviews'.split(),axis=1,inplace=True)
test_df.drop('ratings	reviews'.split(),axis=1,inplace=True)

In [ ]:
train_df = pd.concat([train_df,train_df1],axis=1)
test_df = pd.concat([test_df,test_df1],axis=1)

In [ ]:
train_df.shape,test_df.shape

In [ ]:
train_df['Price']

In [ ]:
y = train_df['Price'].iloc[:,0]
train_df.drop('Price',axis=1,inplace=True)
test_df.drop('Price',axis=1,inplace=True)
train_y = np.log1p(y)
X = train_df.copy()
y=np.log1p(y)
X_test = test_df.copy()
train_df.shape,test_df.shape

In [ ]:
set(train_df.columns) - set(test_df.columns)

In [ ]:
dev_X, val_X, dev_y, val_y = model_selection.train_test_split(X, y, test_size=0.25, random_state=42)

random_seed = 2019
k = 5
fold = list(KFold(k, shuffle = True, random_state = random_seed).split(train))
np.random.seed(random_seed)
from datetime import datetime

print(dev_X.shape,val_X.shape,test_df.shape,val_y.shape,dev_y.shape)

In [ ]:
# Training the model #
pred_test, model, pred_val = run_lgb(dev_X, dev_y, val_X, val_y, X_test)

In [ ]:
k =[] 
arr=model.feature_importance()
for i in range(len(arr)):
    if arr[i] !=0:
        k.append(i)

len(k)

In [ ]:
k_var = np.array(X.columns)
k_var[k].shape

In [ ]:
k_var = list(k_var[k])

In [ ]:
X = X[k_var]
X_test = X_test[k_var]
X.shape,X_test.shape

In [ ]:
X = X.values
X_test = X_test.values

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn import model_selection, preprocessing, metrics
import matplotlib.pyplot as plt
import seaborn as sns
import os

lgb_params1 ={'learning_rate': 0.04, 'max_depth': 15, 'min_child_samples': 8, 'num_leaves': 55,"objective" : "regression", "metric" : "rmse"}


run_lgb = True
print('LGB : ', run_lgb)
# modeling
#--------------------------------------------------------------------------
if run_lgb:
    import lightgbm as lgb
    def kfold_lgb_xgb():
        folds = KFold(n_splits=5, shuffle=True, random_state=7)
        
        oof_lgb = np.zeros(len(X))
        predictions_lgb = np.zeros(len(test_df))

        for fold_, (trn_idx, val_idx) in enumerate(folds.split(X)):
            trn_data = lgb.Dataset(X[trn_idx], label=y[trn_idx])
            val_data = lgb.Dataset(X[val_idx], label=y[val_idx])
            
            print("LGB " + str(fold_) + "-" * 50)
            num_round = 30000
            clf = lgb.train(lgb_params1, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=1000, early_stopping_rounds = 100)
            oof_lgb[val_idx] = clf.predict(X[val_idx], num_iteration=clf.best_iteration)
            
            predictions_lgb += clf.predict(X_test, num_iteration=clf.best_iteration) / folds.n_splits
        

        x = []
        for i in oof_lgb:
            if i < 0:
                x.append(0.0)
            else:
                x.append(i)
        cv_lgb = mean_squared_error(x, y)**0.5
        cv_lgb = str(cv_lgb)
        cv_lgb = cv_lgb[:10]
        
        pd.DataFrame({'preds': x}).to_csv('lgb_oof_' + cv_lgb + '.csv', index = False)
        
        print("CV_LGB : ", cv_lgb)
        return cv_lgb, predictions_lgb
        
    cv_lgb, lgb_ans = kfold_lgb_xgb()
    x = []
    for i in lgb_ans:
        if i < 0:
            x.append(0.0)
        else:
            x.append(i)
    np.save('lgb_ans.npy', x)
    test = ss.copy()
    test['Price'] = np.round(np.expm1(x))
    test.to_csv('lgb_' + cv_lgb + '.csv',index=False)
    from pandas import ExcelWriter
    writer = ExcelWriter('MH_v3_lgb.xlsx')
    test[['Price']].to_excel(writer,'Sheet1')
    writer.save()